# Implementing a Viterbi Decoder and Evaluation for Sequence Labeling

In this assignment, you will build a Viterbi decoder for an LSTM named-entity recognition model. As we mentioned in class, recurrent and bidirectional recurrent neural networks, of which LSTMs are the most common examples, can be used to perform sequence labeling. Although these models encode information from the surrounding words in order to make predictions, there are no "hard" constraints on what tags can appear where.

There hard constraints are particularly important for tasks that label spans of more than one token. The most common example of a span-labeling task is named-entity recognition (NER). As described in Eisenstein, Jurafksy & Martin, and other texts, the goal of NER is to label spans of one or more words as _mentions_ of an _entity_, such as a person, location, organization, etc.

The most common approach to NER is to reduce it to a sequence-labeling task, where each token in the input is labeled either with an `O`, if it is "outside" any named-entity span, or with `B-TYPE`, if it is the first token in an entity of type `TYPE`, or with `I-TYPE`, if it is the second or later token in an entity of type `TYPE`. Distinguishing between the first and later tokens of an entity allow us to identify distinct entity spans even when they are adjacent.

Common values of `TYPE` include `PER` for person, `LOC` for location, `DATE` for date, and so on. In the dataset we load below, there are 17 distinct types.

The span-labeling scheme just described implies that the labels on tokens must obey certain constraints: the tag `I-PER` must follow either `B-PER` or another `I-PER`. It cannot follow `O`, `B-LOC`, or `I-LOC`, i.e., a tag for a different entity type. By themselves, LSTMs or bidirectional LSTMs cannot directly enforce these constraints. This is one reason why conditional random fields (CRFs), which _can_ enforce these constraints, are often layered on top of these recurrent models.

In this assignment, you will implement the simplest possible CRF: a CRF so simple that it does not require any training. Rather, it will assign weight 1 to any sequence of tags that obeys the constraints and weight 0 to any sequence of tags that violates them. The inputs to the CRF, which are analogous to the emission probabilities in an HMM, will come from an LSTM.

But first, in order to test your decoder, you will also implement some functions to evaluate the output of an NER system according to two metrics:
1. You will count the number of _violations_ of the NER label constraints, i.e., how many times `I-TYPE` follows `O` or a tag of a different type or occurs at the beginning of a sentence. This number will be greater than 0 in the raw LSTM output, but should be 0 for your CRF output.
1. You will compute the _span-level_ precision, recall, and F1 of NER output. Although the baseline LSTM was trained to achieve high _token-level_ accuracy, this metric can be misleadingly high, since so many tokens are correctly labeled `O`. In other words, what proportion of spans predicted by the model line up exactly with spans in the gold standard, and what proportion of spans in the gold standard were predicted by the model? For more, see the original task definition: https://www.aclweb.org/anthology/W03-0419/.

We start with loading some code and data and the describe your tasks in more detail.

## Set Up Dependencies and Definitions

In [0]:
#!pip install spacy==2.1.0 allennlp
!pip install --upgrade spacy allennlp
import spacy
print(spacy.__version__)

In [0]:
from typing import Iterator, List, Dict
import torch
import torch.optim as optim
import numpy as np
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor
from allennlp.data.dataset_readers import conll2003

torch.manual_seed(1)

In [0]:
class LstmTagger(Model):
  def __init__(self,
               word_embeddings: TextFieldEmbedder,
               encoder: Seq2SeqEncoder,
               vocab: Vocabulary) -> None:
    super().__init__(vocab)
    self.word_embeddings = word_embeddings
    self.encoder = encoder
    self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                      out_features=vocab.get_vocab_size('labels'))
    self.accuracy = CategoricalAccuracy()

  def forward(self,
              tokens: Dict[str, torch.Tensor],
              metadata,
              tags: torch.Tensor = None) -> Dict[str, torch.Tensor]:
    mask = get_text_field_mask(tokens)
    embeddings = self.word_embeddings(tokens)
    encoder_out = self.encoder(embeddings, mask)
    tag_logits = self.hidden2tag(encoder_out)
    output = {"tag_logits": tag_logits}
    if tags is not None:
      self.accuracy(tag_logits, tags, mask)
      output["loss"] = sequence_cross_entropy_with_logits(tag_logits, tags, mask)

    return output

  def get_metrics(self, reset: bool = False) -> Dict[str, float]:
    return {"accuracy": self.accuracy.get_metric(reset)}

## Import Data

In [0]:
reader = conll2003.Conll2003DatasetReader()
train_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.train.ner.sample'))
validation_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.development.ner.sample'))

vocab = Vocabulary.from_instances(train_dataset + validation_dataset)

159377B [00:00, 1953327.61B/s]
562it [00:00, 4708.35it/s]
8366B [00:00, 13211425.93B/s]
23it [00:00, 3474.11it/s]
100%|██████████| 585/585 [00:00<00:00, 47950.36it/s]


In [0]:
vocab

Vocabulary with namespaces:  tokens, Size: 1931 || labels, Size: 34 || Non Padded Namespaces: {'*tags', '*labels'}

## Define and Train Model

In [0]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, bidirectional=False, batch_first=True))
model = LstmTagger(word_embeddings, lstm, vocab)
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
# optimizer = optim.AdamW(model.parameters(), lr=1e-4, eps=1e-8)
optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2, sorting_keys=[("tokens", "num_tokens")])
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=100,
                  cuda_device=cuda_device)
trainer.train()

{'best_epoch': 98,

 'best_validation_accuracy': 0.8795918367346939,

 'best_validation_loss': 0.36728941490097594,
 
 'epoch': 99,
 
 'peak_cpu_memory_MB': 2600.008,
 
 'peak_gpu_0_memory_MB': 541,
 
 'training_accuracy': 0.9259612329202415,
 
 'training_cpu_memory_MB': 2600.008,
 
 'training_duration': '0:02:16.681880',
 
 'training_epochs': 99,
 
 'training_gpu_0_memory_MB': 541,
 
 'training_loss': 0.19139084464135445,
 
 'training_start_epoch': 0,
 
 'validation_accuracy': 0.8775510204081632,
 
 'validation_loss': 0.3792670750020382}

## Evaluation

The simple code below creators a `predictor` object that applies the model to an input example and then loops over the examples in the validation set, printing out the input token, gold-standard output, and model output. You can see from these methods how to access data and model outputs for evaluation.

In [0]:
predictor = SentenceTaggerPredictor(model, dataset_reader=reader)

def tag_sentence(s):
  tag_ids = np.argmax(predictor.predict_instance(s)['tag_logits'], axis=-1)
  fields = zip(s['tokens'], s['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
  return list(fields)

baseline_output = [tag_sentence(i) for i in validation_dataset]
training_output = [tag_sentence(i) for i in train_dataset]

In [0]:
#(token, tag, predicted tag)
print(baseline_output)
len(baseline_output)

[[(With, 'O', 'O'), (a, 'O', 'O'), (wave, 'O', 'O'), (of, 'O', 'O'), (his, 'O', 'O'), (hand, 'O', 'O'), (,, 'O', 'O'), (Peng, 'B-PERSON', 'B-PERSON'), (Dehuai, 'I-PERSON', 'I-ORG'), (said, 'O', 'O'), (that, 'O', 'O'), (despite, 'O', 'O'), (being, 'O', 'O'), (over, 'O', 'O'), (100, 'B-CARDINAL', 'B-CARDINAL'), (regiments, 'O', 'O'), (,, 'O', 'O'), (let, 'O', 'O'), ('s, 'O', 'O'), (call, 'O', 'O'), (this, 'O', 'O'), (campaign, 'O', 'O'), (the, 'B-EVENT', 'O'), (Hundred, 'I-EVENT', 'I-EVENT'), (Regiments, 'I-EVENT', 'I-EVENT'), (Offensive, 'I-EVENT', 'I-EVENT'), (., 'O', 'O')], [(That, 'B-TIME', 'O'), (afternoon, 'I-TIME', 'I-DATE'), (,, 'O', 'O'), (Peng, 'B-PERSON', 'B-PERSON'), (Dehuai, 'I-PERSON', 'I-DATE'), (first, 'B-ORDINAL', 'I-DATE'), (used, 'O', 'O'), (the, 'O', 'O'), (name, 'O', 'O'), (Hundred, 'B-EVENT', 'I-EVENT'), (Regiments, 'I-EVENT', 'I-EVENT'), (Offensive, 'I-EVENT', 'I-EVENT'), (in, 'O', 'O'), (telegrams, 'O', 'O'), (sent, 'O', 'O'), (to, 'O', 'O'), (various, 'O', 'O'), 

23

In [0]:
len(baseline_output[0])

27

Now, you can implement two evaluation functions: `violations` and `span_stats`.

In [0]:
import re

In [0]:
# TODO: count the number of NER label violations,
# such as O followed by I-TYPE or B-TYPE followed by
# I-OTHER_TYPE
# Take tagger output as input
def violations(tagged):
  count = 0
  for sentence in tagged:
    for i in range(len(sentence)-1):
      if i == 0 and sentence[i][2].split('-')[0] == 'I': #if the starting tag itself is I-Type then it is violation
        count += 1
      if sentence[i][2] == 'O' and sentence[i+1][2].split('-')[0] == 'I':
        count += 1
      if sentence[i][2].split('-')[0] == 'B' and sentence[i+1][2].split('-')[0] == 'I':
        if sentence[i][2].split('-')[1] != sentence[i+1][2].split('-')[1]:
          count += 1
    #print((sentence))
  return count

## You can check how many violations are made by the model output in predictor.
print("Voilations for validation data is ", violations(baseline_output))
print("Voilations for training data is ", violations(training_output))

Voilations for validation data is  27
Voilations for training data is  273


In [0]:
# TODO: return the span-level precision, recall, and F1
# Take tagger output as input
def span_stats(tagged):
  def get_entities(tagged, sys):
   #sys =1 for goldstandard output, sys=2 for predicted output
   spans = []  #list of spans
   prev_tag = 'O' #initial 'O' for checking condition
   prev_type = ''
   begin_offset = 0
   for sentence in tagged:
    for i,seq in enumerate(sentence):
      tag = seq[sys].split('-')[0]
      try:
        tag_type = seq[sys].split('-')[1]
      except:
        tag_type = seq[sys].split('-')[0] #just use 'O' as type
      if prev_tag == 'B' and tag == 'B': 
        spans.append((prev_type, begin_offset, i-1))
      if prev_tag == 'B' and tag == 'O': 
        spans.append((prev_type, begin_offset, i-1)) 
      if prev_tag == 'I' and tag == 'B': 
        spans.append((prev_type, begin_offset, i-1)) 
      if prev_tag == 'I' and tag == 'O': 
        spans.append((prev_type, begin_offset, i-1))
      if prev_tag == 'B' and tag == 'I' and tag_type != prev_type: 
        spans.append((prev_type, begin_offset, i-1))
      if prev_tag == 'I' and tag == 'I' and tag_type != prev_type: 
        spans.append((prev_type, begin_offset, i-1))
      if tag == 'B':
        begin_offset = i  
      if prev_tag == 'O' and tag == 'I':
        begin_offset = i 
      if prev_tag != 'O' and prev_type != tag_type:
        begin_offset = i
      prev_tag = tag
      prev_type = tag_type
   return spans
  spans = get_entities(tagged, 2)
  spans_actual = get_entities(tagged, 1)
  #system_output = set(spans)
  #gold_satndard = set(spans_actual)
  #true_positive = len(system_output & gold_satndard)
  true_positive = sum(el in spans for el in spans_actual)
  pred_n = len(spans)
  actual_n = len(spans_actual)
  print("Number of True Positives/matches in both predicted output and gold standard:",true_positive)
  print("Number of NER spans in predicted output:",pred_n)
  print("Number of NER spans in gold standard:", actual_n)
  precision = true_positive/pred_n if pred_n > 0 else 0
  recall = true_positive/actual_n if actual_n > 0 else 0
  f1 = (2 * precision * recall) / (precision + recall) if precision+recall > 0 else 0
  return {'precision': precision,
          'recall': recall,
          'f1': f1}

span_stats(baseline_output)

Number of True Positives/matches in both predicted output and gold standard: 12
Number of NER spans in predicted output: 61
Number of NER spans in gold standard: 43


{'f1': 0.23076923076923075,
 'precision': 0.19672131147540983,
 'recall': 0.27906976744186046}

In [0]:
span_stats(training_output)

Number of True Positives/matches in both predicted output and gold standard: 549
Number of NER spans in predicted output: 1018
Number of NER spans in gold standard: 808


{'f1': 0.6013143483023001,
 'precision': 0.5392927308447937,
 'recall': 0.6794554455445545}

In [0]:
!pip install seqeval
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
labels_bio = []
pred_bio = []
for sentence in baseline_output:
  tmp_1 = []
  tmp_2 = []
  for i,seq in enumerate(sentence):
    #print(seq[1])
    tmp_1.extend([seq[1]])
    tmp_2.extend([seq[2]])
  labels_bio.append(tmp_1)
  #print(tmp_1)
  pred_bio.append(tmp_2)
#print(labels_bio) 
#pred_bio
print(classification_report(labels_bio, pred_bio))

The result obtained from span_stats matches with the seqeval package results.

**Validation Set Results:**

F1: 0.23

Precision: 0.20

Recall: 0.28

**Training Set Results:**

F1: 0.60

Precision: 0.54

Recall: 0.68

## Decoding

Now you can finally implement the simple Viterbi decoder. The `predictor` object, when applied to an input sentence, first calculates the scores for each possible output tag for each token. See the line `predictor.predict_instance(i)['tag_logits']` in the code above.

Then, you will construct a transition matrix. You can use the code below to get a list of the tags the model knows about. For a set of K tags, construct a K-by-K matrix with a log(1)=0 when a transition between a given tag pair is valid and a log(0)=-infinity otherwise.

Finally, implement a Viterbi decoder that takes the predictor object and a dataset object and outputs tagged data, just like the `tag_sentence` function above. It should use the Viterbi algorithm with the (max, plus) semiring. You'll be working with sums of log probabilities instead of products of probabilties.

Run your `violations` function on the output of this decoder to make sure that there are no invalid tag transitions. Also, compare the span-level metrics on `baseline_output` and your new output using your `span_stats` function.

In [0]:
# This code show how to map from output vector components to labels
print(vocab.get_index_to_token_vocabulary('labels'))



{0: 'O', 1: 'B-GPE', 2: 'I-ORG', 3: 'I-DATE', 4: 'B-CARDINAL', 5: 'I-EVENT', 6: 'B-PERSON', 7: 'B-NORP', 8: 'B-DATE', 9: 'B-ORG', 10: 'B-LOC', 11: 'I-LOC', 12: 'I-FAC', 13: 'I-PERSON', 14: 'I-GPE', 15: 'I-CARDINAL', 16: 'B-EVENT', 17: 'I-TIME', 18: 'I-WORK_OF_ART', 19: 'B-ORDINAL', 20: 'B-FAC', 21: 'B-TIME', 22: 'I-LAW', 23: 'I-QUANTITY', 24: 'I-NORP', 25: 'I-MONEY', 26: 'B-MONEY', 27: 'B-WORK_OF_ART', 28: 'B-QUANTITY', 29: 'B-LAW', 30: 'B-PRODUCT', 31: 'I-PRODUCT', 32: 'B-PERCENT', 33: 'I-PERCENT'}


In [0]:
import numpy as np
#import math

In [0]:
K = 34 #number of tags

trans_score = np.array([[np.log(1) for j in range(K)] for i in range(K)]) #transition matrix

for i in range(34):
  for j in range(34):
    if vocab.get_token_from_index(j, 'labels').split('-')[0] == 'I':
      trans_score[i,j] = np.log(0)
    if vocab.get_token_from_index(i, 'labels').split('-')[0] == 'O' and vocab.get_token_from_index(j, 'labels').split('-')[0] == 'I':
      trans_score[i,j] = np.log(0) 
    if vocab.get_token_from_index(i, 'labels').split('-')[0] == 'B' and vocab.get_token_from_index(j, 'labels').split('-')[0] == 'I' and vocab.get_token_from_index(i, 'labels').split('-')[1] != vocab.get_token_from_index(j, 'labels').split('-')[1]:
      trans_score[i,j] = np.log(0)
    if vocab.get_token_from_index(i, 'labels').split('-')[0] == 'I' and vocab.get_token_from_index(j, 'labels').split('-')[0] == 'I' and vocab.get_token_from_index(i, 'labels').split('-')[1] != vocab.get_token_from_index(j, 'labels').split('-')[1]:
      trans_score[i,j] = np.log(0)
 
start_score = np.array([np.log(0) if vocab.get_token_from_index(i, 'labels').split('-')[0]=='I' else np.log(1) for i in range(34)]) #for the initial transition
#print(vocab.get_token_from_index(i, 'labels'))

In [0]:
#Each prob here is in log space
#since ending scores/probabilities are same for all tags, only considering start/initial probability
#initial prob or score would be -inf for I-TYPE, and for others it is 0 (log1)
predictor = SentenceTaggerPredictor(model, dataset_reader=reader)
def tag_viterbi(s):
  emission_weights = np.array(predictor.predict_instance(s)['tag_logits'])
  N = emission_weights.shape[0] #number of tokens
  scores = np.array([[np.log(1) for j in range(N)] for i in range(K)]) #to store scores of the trellis
  ptr = np.array([[-1 for j in range(N)] for i in range(K)])  #to keep back pointer
  for i in range(N):
        if i == 0:
            scores[:,0] = np.add(start_score, emission_weights[i])
        else:
            for j in range(K):
                scores[j,i]= np.max(np.add(trans_score[:,j] , scores[:,i-1])) + emission_weights[i,j]
                ptr[j,i] = np.argmax(np.add(trans_score[:,j] , scores[:,i-1]))
  indx = np.argmax(scores[:, i])
  #print(scores.shape)
  #print(ptr.shape)
  #print(indx)
  tag_ids = []
  while indx != -1:
        tag_ids.append(indx)
        indx = ptr[indx,i]
        i -= 1
  tag_ids.reverse() #reverse backpointer to get list of indices to tags
  fields = zip(s['tokens'], s['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
  return list(fields)

In [0]:
viterbi_output = [tag_viterbi(i) for i in validation_dataset]
viterbi_train_output = [tag_viterbi(i) for i in train_dataset]
print("Violations for validation data after viterbi is ", violations(viterbi_output))
print("Violations for training data after viterbi is ", violations(viterbi_train_output))

Violations for validation data after viterbi is  0
Violations for training data after viterbi is  0


**0 violations with Viterbi algorithm on top of CRF scores**

In [0]:
print("Evaluation for validation data: ", span_stats(viterbi_output))
print("Evaluation for training data: ",span_stats(viterbi_train_output))

Number of True Positives/matches in both predicted output and gold standard: 9
Number of NER spans in predicted output: 80
Number of NER spans in gold standard: 43
Evaluation for validation data:  {'precision': 0.1125, 'recall': 0.20930232558139536, 'f1': 0.14634146341463414}
Number of True Positives/matches in both predicted output and gold standard: 478
Number of NER spans in predicted output: 1182
Number of NER spans in gold standard: 808
Evaluation for training data:  {'precision': 0.40439932318104904, 'recall': 0.5915841584158416, 'f1': 0.4804020100502512}


The nuber of spans in predicted output increased compared to before viterbi, and hence reduction in true positive and F1 score.